# Loop dimers

In [5]:
# %load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers
running on node: jojo


In [1]:
sys.path.insert(0, "/home/broerman/projects/crispy_shifty")
from crispy_shifty.utils.io import gen_array_tasks

design_list_file = "/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/02_design_filter_3/filtered_designs.list"
output_path = "/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/03_close_loops"

# Format as a string of key value pairs with no leading dashes on the argument names
options = "out:level 100 holes:dalphaball /home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc indexed_structure_store:fragment_store /home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5 precompute_ig true"

gen_array_tasks(distribute_func='crispy_shifty.protocols.looping.loop_dimer',
                design_list_file=design_list_file,
                output_path=output_path,
                queue='short',
                memory='8G',
                nstruct=1,
                nstruct_per_task=50,
                options=options,
                simulation_name='03_close_loops')

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


Run the following command with your desired environment active:
sbatch -a 1-$(cat /home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/03_close_loops/tasks.cmds | wc -l) /home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/03_close_loops/run.sh


In [ ]:
from crispy_shifty.utils.io import collect_score_file
output_path = "/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/03_close_loops"
collect_score_file(output_path, 'scores')

## Everything after this is for testing ##

In [1]:
import pyrosetta
from pyrosetta.distributed.cluster.io import IO
print(IO.REMARK_FORMAT)
IO.REMARK_FORMAT = 'REMARK CrispyShifty: '
print(IO.REMARK_FORMAT)
path = '/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/02_design_filter_3/decoys/0006/_5fc0cbb34e88485690c5fd78d411902d.pdb.bz2'
scores = pyrosetta.distributed.cluster.get_scores_dict(path)["scores"]

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


REMARK PyRosettaCluster: 
REMARK CrispyShifty: 


In [1]:
import pyrosetta
import pyrosetta.distributed.cluster as cluster
path = '/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/03_close_loops_test/decoys/0000/03_close_loops_8bf01523165b483b9193640cfdc02091.pdb.bz2'
scores = pyrosetta.distributed.cluster.get_scores_dict(path)["scores"]
print(scores)

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
# Fix the use of REMARK CrispyShifty" instead of "REMARK PyRosettaCluster" 

from glob import iglob
import bz2
# filepath = "/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/02_design_filter_2/decoys/"
# filepath = "/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/02_design_filter_3/decoys/"
# filepath = "/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/03_close_loops_test/decoys/"
for filename in iglob(filepath + '*/*.pdb.bz2'):
    with open(filename, "rb") as fbz2:
        pdbstring = bz2.decompress(fbz2.read()).decode()
    split_pdbstr = pdbstring.split(os.linesep)
    for i, line in enumerate(reversed(split_pdbstr)):
        if line.startswith("REMARK CrispyShifty"):
            newline = line.replace("CrispyShifty", "PyRosettaCluster")
            split_pdbstr[-i-1] = newline
            with(open(filename, "wb")) as fbz2:
                fbz2.write(bz2.compress(os.linesep.join(split_pdbstr).encode()))
            break

In [2]:
sys.path.insert(0, "/home/broerman/projects/crispy_shifty")
from crispy_shifty.utils.io import gen_array_tasks

design_list_file = "/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/02_design_filter_3/filtered_designs (copy).list"
output_path = "/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/testing/03_close_loops_design"

# Format as a string of key value pairs with no leading dashes on the argument names
options = "out:level 100 holes:dalphaball /home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc indexed_structure_store:fragment_store /home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5 precompute_ig true"

gen_array_tasks(distribute_func='crispy_shifty.protocols.looping.loop_dimer',
                design_list_file=design_list_file,
                output_path=output_path,
                queue='short',
                memory='8G',
                nstruct=1,
                nstruct_per_task=1,
                options=options,
                simulation_name='03_close_loops')

Run the following command with your desired environment active:
sbatch -a 1-$(cat /home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/testing/03_close_loops_design/tasks.cmds | wc -l) /home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/testing/03_close_loops_design/run.sh


In [6]:
# testing

sys.path.insert(0, "/home/broerman/projects/crispy_shifty")
from crispy_shifty.protocols.looping import loop_dimer
import pyrosetta.distributed.io as io

pyrosetta.distributed.maybe_init(**{
    "options": "-corrections::beta_nov16 true",
    "extra_options":{
        "-out:level": "100",
        "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
        "-indexed_structure_store:fragment_store": "/home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5",
        "-precompute_ig": "true"
}})

file = '/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/02_design_filter/decoys/0000/02_design_filter_0a0e27de4de64d10b9c4272024e98bf8.pdb.bz2'

pposes = loop_dimer(**{'pdb_path': file})

for ppose in pposes:
    pose = io.to_pose(ppose)
    pose.dump_pdb(f'testing/loop_dimer_test_scored.pdb')

0.03 min: Attempting closure by loop match...success.
0.06 min: Scoring...complete.
0.10 min: Attempting closure by loop match...


ERROR: No loop closures found below threshold, exiting
ERROR:: Exit from: /home/benchmark/rosetta/source/src/protocols/pose_length_moves/ConnectChainsMover.cc line: 221


failed.
0.11 min: Attempting closure by loop remodel...      4.574999999999999       9.712999999999999      -8.512999999999998
success.
0.50 min: Scoring...complete.


In [ ]:
import bz2
with open('testing/unloopable.pdb', 'rb') as f_in:
    with open('testing/unloopable.pdb.bz2', 'wb') as f_out:
        f_out.write(bz2.compress(f_in.read()))

In [8]:
# testing

sys.path.insert(0, "/home/broerman/projects/crispy_shifty")
from crispy_shifty.protocols.looping import loop_dimer
import pyrosetta.distributed.io as io

pyrosetta.distributed.maybe_init(**{
    "options": "-corrections::beta_nov16 true",
    "extra_options":{
        "-out:level": "100",
        "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
        "-indexed_structure_store:fragment_store": "/home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5",
        "-precompute_ig": "true"
}})

file = 'testing/unloopable.pdb.bz2'

pposes = loop_dimer(**{'pdb_path': file})

for ppose in pposes:
    pose = io.to_pose(ppose)
    pose.dump_pdb(f'testing/unloopable_dimer_test.pdb')

0.08 min: Attempting closure by loop match...


ERROR: No loop closures found below threshold, exiting
ERROR:: Exit from: /home/benchmark/rosetta/source/src/protocols/pose_length_moves/ConnectChainsMover.cc line: 221


failed.
4.23 min: Attempting closure by loop remodel...      52.65600000000000       39.76900000000000       35.24300000000000
failed.
